# Migration - Local Area

-----

### Requirements



#### Observations & Dimensions

We're taking 5 tabs (see below table). The `observations` and the `indictor` will vary a little for each tab,

|Tab name| Get obs from columns named | Set 'Indictor' dimension to |
|---|---|---|
| Short Term Migration Flow | Short-Term International Migration Inflow Estimates | Inflow Estimates
| Non-UK Born Population | Resident Population | Not UK Born Population
| Non British Population | Resident Population |  Not British Population
| NINo Registrations | Migrant NINo Registrations | NINo Registrations
| GP Registrations | New Migrant GP Registrations | New Gp Registrations


The required dimensions are:

* **Geography** - use the area code column
* **Time** - from the top and just the year, so "Jan 2009 to Dec 2009" == 2009
* **Indictor** - as per above table

-----
    
Notes:

It's always worth getting the file out of /sources and having a look over.

In [1]:
%cd mock-transformations/

/workspace/mock-transformations


In [2]:
from databaker.framework import *

# the Seasonally Adjusted Tabs
sa_tabs = loadxlstabs("./sources/lamisspreadsheet.xls")

Loading ./sources/lamisspreadsheet.xls which has size 1138688 bytes
Table names: ['Notes', 'Migration Flows', 'Short-Term Migration Inflows', 'Non-UK Born Population', 'Non-British Population', 'NINo Registrations', 'GP Registrations', 'Births to Non-UK Born Mothers']


In [3]:
# This is the test I need to do to see how merged cells behave

sa_tabs[2].excel_ref("C1").expand(RIGHT).is_not_blank()

# So it looks like this is a case of CLOSEST, ABOVE, and will need to substring after the fact in pandas

{<K1 'Mid-2012 to Mid-2013'>, <M1 'Mid-2013 to Mid-2014'>, <E1 'Mid-2009 to Mid-2010 '>, <I1 'Mid-2011 to Mid-2012'>, <C1 'Mid-2008 to Mid-2009 '>, <S1 'Mid-2016 to Mid-2017'>, <Q1 'Mid-2015 to Mid-2016'>, <G1 'Mid-2010 to Mid-2011'>, <O1 'Mid-2014 to Mid-2015'>}

In [25]:
work = {'test': 1}
work

{'test': 1}

In [35]:
for tab in sa_tabs:
    if tab.name.split()[-1] in ("Inflows", "Registrations"):
        print("😍 " + tab.name)
        indicator = tab.excel_ref("C2").expand(RIGHT).is_not_blank().regex("(Inflow)|(Registrations)")
    elif tab.name.split()[-1] == "Population":
        print("🥰 " + tab.name)
        indicator = tab.excel_ref("C3").expand(RIGHT).is_not_blank().regex("Population")
    else:
        # print("😖 " + tab.name)
        continue
    
    work[tab.name] = tab.name

    time = tab.excel_ref("C1").expand(RIGHT).is_not_blank()
    geography = tab.excel_ref("A4").expand(DOWN).is_not_blank()
    observations = geography.waffle(indicator)

    dimensions = [HDim(geography, "Geography", DIRECTLY, LEFT),
                  HDim(time, "Year", CLOSEST, ABOVE),
                  HDim(indicator, "Indicator", DIRECTLY, ABOVE)]

    work[tab.name] = ConversionSegment(tab, dimensions, observations).topandas()

😍 Short-Term Migration Inflows



AssertionError: 

In [34]:
work['Short-Term Migration Inflows'].topandas()

AssertionError: 